In [1]:
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def tensorboard_summaries(conv_layers=None, dense_layers=None):
    if conv_layers is not None:
        for cl_name, cl in conv_layers.items():
            shape = cl.shape.as_list()
            for f in range(shape[3]):
                tf.summary.image("{}/filter/{}".format(cl_name, f), tf.reshape(cl[:, :, :, f], [tf.shape(cl)[0], shape[1], shape[2], 1]))
                
    if dense_layers is not None:
        for dl_name, dl in dense_layers.items():
           tf.summary.histogram("{}/weights".format(dl_name), dl) 

def cnn_model_fn(features, labels, mode):
    # reshape input layer to (N, 28, 28, 1)
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1]) 

    # conv layer 1, output is (N, 28, 28, 32)
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu, name="conv1" )

    # pool layer 1, output is (N, 14, 14, 32)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2, name="pool1")

    # conv layer 2, output is (N, 14, 14, 64)
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5,5], padding="same", activation=tf.nn.relu, name="conv2")

    # pool layer 2, output is (N, 7, 7, 64)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2,  name="pool2")

    # flatten to accept dense layer (N, 7*7*64)
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # dense layer
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, name="dense")

    # apply dropout to help regularization
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training = mode == tf.estimator.ModeKeys.TRAIN)

    # logits layer
    logits = tf.layers.dense(inputs=dropout, units=10, name="logits")
    
    # tensorboard summary for weights and variables in the layers
    tensorboard_summaries(conv_layers={"conv1": conv1, "conv2": conv2}, 
                         dense_layers={"dense": dense, "logits": logits })    

    # predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1), 
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        # wrap predictions into a class and return EstimatorSpec object
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # one-hot encoding
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    
    # minimize on cross-entropy
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)  # loss is a scalar tensor
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [ ]:
import datetime

d = datetime.datetime.now()

# load data
mnist = tf.keras.datasets.mnist.load_data(path='mnist.pz')  ## (x_train, y_train), (x_test, y_test)
train_data = np.asarray(mnist[0][0], dtype=np.float32)
train_labels = np.asarray(mnist[0][1], dtype=np.int32)
eval_data = np.asarray(mnist[1][0], dtype=np.float32)
eval_labels = np.asarray(mnist[1][1], dtype=np.int32)

# create the estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, 
                                          model_dir="/mnt/data/tensorboard/mnist_convnet_model_{}".format(d.strftime("%Y%M%dT%H%m%S")))

# setup a logging hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

# train
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":train_data}, 
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(input_fn=train_input_fn, steps=20000) #, hooks=[logging_hook])

# evaluate
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":eval_data}, 
    y=eval_labels,
    num_epochs=1, 
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/mnt/data/tensorboard/mnist_convnet_model_20183529T140513', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f62b0bc44a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /mnt/data/tensor